# Run the following cells to see the live detection of facial emotions

In [1]:
import cv2
import numpy as np
import tensorflow as tf

# When you are using the CNN Model without "Disgust" , you have to choose the right   class_names Array

In [2]:
#classes = ["0","1","2","3","4","5","6"]
#class_names = ["angry","disgust","fear","happy","neutral","sad","surprised"]
class_names = ["angry","fear","happy","neutral","sad","surprised"]
classes = ["0","1","2","3","4","5"]

model = tf.keras.models.load_model('models&Historys/model/CNN_With_Generator_No_Disgust_model.h5')


Metal device set to: Apple M1


2022-05-16 22:26:31.216919: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-16 22:26:31.218529: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
def closeAll():
    cv2.namedWindow("win")
    cv2.startWindowThread()
    cv2.destroyAllWindows()


In [4]:
#thest the model on one image
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
frame = cv2.imread("test.png")
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

faces = faceCascade.detectMultiScale(gray, 1.1, 4)
for x, y, w, h in faces:
    roi_gray = gray[y:y + h, x: x + w]
    roi_color = frame[y:y + h, x: x + w]
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 11, 55), 2)
    facess = faceCascade.detectMultiScale(roi_gray)
    if len(facess) == 0:

        print("Face not detected")
    else:
        for (ex, ey, ew, eh) in facess:  #show found faces and the predicted emotions
            face_roi = roi_color[ey: ey + eh, ex: ex + ew]
            final_image = cv2.resize(face_roi, (160, 160))
            final_image = np.expand_dims(final_image, axis=0)  #need 4th Dimention
            #final_image = final_image / 255.0  #normalize
            predictions = model.predict(final_image)
            print(class_names[np.argmax(predictions)])



happy


2022-05-16 22:26:31.648909: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-05-16 22:26:31.720227: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [5]:
# Run the live cam
font = cv2.FONT_HERSHEY_PLAIN
#set the rectangle background to white
rectangle_bgr = (255, 255, 255)
#make a black image
img = np.zeros((500, 500))
#set some text
text = "Some text in a box!"
# get the width and height of the text box
(text_width, text_height) = cv2.getTextSize(text, font, fontScale=1.5, thickness=1)[0]
# set the text start position
text_offset_x = 10
text_offset_y = img.shape[0] - 25
#make the coords of the box with a small padding of two pixels
box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height - 2))
cv2.rectangle(img, box_coords[0], box_coords[1], rectangle_bgr, cv2.FILLED)
cv2.putText(img, text, (text_offset_x, text_offset_y), font, fontScale=1.5, color=(0, 0, 0), thickness=1)
cap = cv2.VideoCapture(1)
# Check if the webcam is opened correctly
if not cap.isOpened():
    cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise IOError("Cannot open webcam")


while True:
    ret, frame = cap.read()
    #eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #print(faceCascade.empty())
    faces = faceCascade.detectMultiScale(gray,1.1,4)
    for x,y,w,h in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        facess = faceCascade.detectMultiScale(roi_gray)
        if len(facess) == 0:
            continue
        else:
            for (ex,ey,ew,eh) in facess:
                face_roi = roi_color[ey: ey+eh, ex:ex + ew] ## cropping the face
                final_image = cv2.resize(face_roi, (160,160))
                final_image = np.expand_dims(final_image,axis=0) ## need fourth dimension # we dont rescale because the model have a rescaling layer
                font = cv2.FONT_HERSHEY_SIMPLEX
                Predictions = model.predict(final_image)
                #font = cv2.FONT_HERSHEY_PLAIN


                #show the name of predicted emotion
                status = class_names[np.argmax(Predictions)]
                x1,y1,w1,h1 = 0,0,175,75
                #Draw black background rectangle
                cv2.rectangle(frame, (x1, x1), (x1 + w1, y1 + h1), (0,0,0), -1)
                #Addd text
                cv2.putText(frame, status, (x1 + int(w1/10), y1 + int(h1/2)), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                cv2.putText(frame, status,(100,150),font, 3,(0, 0, 255),2,cv2.LINE_4)
                cv2.rectangle(frame, (x,y), (x+w, y+h), (0, 0, 255))

    cv2.imshow('Face Emotion Recognition', frame)
    if cv2.waitKey(2) & 0xFF == ord('q') :
        break
cap.release()
cv2.destroyAllWindows()


OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!


In [8]:
closeAll() # if the python3.80 window did not close, use this cell separately